In [1]:
from CONSTANTS import *
import torch
import numpy as np

def _save_missing_indeces(start, attributes, missing_indeces, is_missing_indeces=False):
    if "index" not in attributes.keys():
        return
    else:
        # Compute missing values from current processing step
        if not is_missing_indeces:
            upper_value = torch.max(attributes["index"]).item()
            complete_tensor_of_indeces = torch.arange(start, upper_value + 1).detach()
            missing_indeces = list(set(missing_indeces + np.setdiff1d(complete_tensor_of_indeces.cpu().numpy(), attributes["index"].cpu().numpy()).tolist()))
        else:
            upper_value = start - 1
            missing_indeces = np.setdiff1d(np.array(missing_indeces),
                                                 attributes["index"].cpu().numpy()).tolist()
        # Dump information in file for future use. The information consists of two lines:
        # 1. Indeces of samples that have been skipped.
        # 2. Index of the last sample that has been processed plus 1.
        with open(FILE_MISSING_INDECES, "w") as f:
            f.write(', '.join(map(str, missing_indeces)) + "\n")
            f.write(str(upper_value + 1))



attributes = {"index": torch.tensor([i for i in range(35, 151)])}     
missing_indeces = [2,6,36, 45]
_save_missing_indeces(151, attributes, missing_indeces, True)

In [10]:
import pinecone

# Create reference to pinecone index
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)
index = pinecone.Index(PINECONE_INDEX_NAME)

index.delete(ids=[str(i) for i in range(3000, 4000)])

{}